<a href="https://colab.research.google.com/github/XavierCachan/moduleIA_S4/blob/main/Optimisation_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP IA Partie 1 - Quelques notions d'optimisation - IUT de Cachan 2024**
XM - Février 2024 - Version : 0.8

-----

Note : Pour avancer dans ce notebook, il suffit d'exécuter (petite flèche), ou compléter puis exécuter, les différents blocs de code placés ci-dessous.

Cette première partie a pour objectif de faire comprendre sur un exemple très simple les principes d'une optimisation déterministe par descente de gradient.

On considère un récipient ayant la forme d'une parabole, et on lâche une bille sur l'un des côtés du récipient. On cherche à calculer quelle sera la position finale de la bille (ce problème peut bien sûr se résoudre par un calcul analytique, mais si la forme du récipient était plus complexe, le challenge serait plus difficile).

**1. Import des modules nécessaires pour les calculs et l'affichage**

In [ ]:
import numpy as np                # Module de fonctions mathématiques
import math                       # Module de fonctions mathématiques
import matplotlib.pyplot as plt   # Module de fonctions pour affichage

**2. Précisions sur le problème posé**

On considère le récipient suivant :

In [ ]:
N = 40                            # Nombre de points considérés pour le dessin
Xv = np.linspace(0,4,num=N)       # Fabrication d'un vecteur de N points régulièrement espacés entre 0 et 4 (0, 0.1, 0.2, ... 3.9)

# Définition de la fonction parabolique du récipient
Yv = (Xv-2)**2 + 1

# Affichage du récipient et de la position initiale de la bille
X_init = 0.2      # Position X initiale de la bille
fig,ax = plt.subplots(1,figsize=(8,5))    # Format de l'affichage à réaliser
ax.plot (Xv, Yv, linestyle="-", linewidth=1, label="Récipient")
ax.plot ((0,4), (0,0), linestyle="-", color="black", label="Sol")
ax.plot (X_init, (X_init-2)**2 + 1, marker="x", linestyle="none", color="orange", label="Position initiale de la bille")
ax.plot ((X_init,X_init), (0,(X_init-2)**2 + 1), linestyle="-", color="orange", label="Distance par rapport au sol")
ax.set_title("Forme du récipient")
ax.set_xlabel("Position X")
ax.set_ylabel("Position Y de la bille")
ax.legend()       # Ajout de la legende à l'affichage
plt.show()        # Demande d'affichage


Comment obtenir la position finale de la bille ? Il faut chercher à minimiser la distance entre sa position verticale (Y) et le sol (en physique en toute rigueur on dirait que l'on veut minimiser son énergie potentielle) qui est dessinée en orange sur le graphique.

**3. Recherche du minimum par optimisation**

La variable inconnue est donc X, et on commence par définir la fonction coût qui est ici directement la distance (Y - 0) pour un X donné.

In [ ]:
# Définition de la fonction coût = distance par rapport à l'axe horizontal
def cout(X): # argument X = position de la bille
  return (X-2)**2 + 1

On définit ensuite la fonction retournant la dérivée de cette fonction coût par rapport à la variable X, dérivée que l'on va chercher à annuler pour trouver la distance minimale.

In [ ]:
# Définition de la fonction pente (dérivée) du coût : si l'on trouve la valeur qui l'annule, alors on sera au point de coût minimum
def dcout(X): # argument X = position de la bille
  return 2*(X-2)

**Départ de l'optimisation** : on commence par définir les paramètres :

In [ ]:
taux_evolution = 0.1    # "Vitesse" d'avancée de l'algorithme. Par defaut 0.1
maxIter = 10000         # Nombre max d'itérations. Par defaut 10000
eps = 0.0001            # On cherche à ce que la dérivée devienne plus petite que eps (=> valeur min du coût). Par defaut 0.0001
iter = 0                # Numéro itération en cours

# Initialisation des paramètres de l'optimisation
X_init = 0.2            # Position X initiale de la bille. Par defaut 0.2
Y_init = cout(X_init)   # Distance initiale

grad = dcout(X_init)    # Valeur initiale de la pente

liste_iter = []         # Listes pour affichage de la convergence de l'erreur
liste_erreur = []
liste_dist = []
liste_X = []

Regardons ce qu'il se passe à la première itération :

In [ ]:
gradD = dcout(X_init)                      # Calcul de la pente
grad = math.sqrt(gradD**2)                 # Valeur absolue
X = X_init - taux_evolution * gradD        # On bouge un peu X dans la direction opposée de la dérivée (ici la pente est négative, et on veut augmenter X)
iter += 1

# Sauvegarde pour affichage des courbes de pente et distance vs itérations
liste_iter.append(iter)
liste_erreur.append(grad)
liste_dist.append(cout(X))
liste_X.append(X)

# Analyse de la position obtenue pour notre optimisation à l'itération 1
Y_mod = (X-2)**2 + 1

# Affichage des résultats du modèle
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (Xv, Yv, linestyle="-", linewidth=1, label="Récipient")
ax.plot ((0,4), (0,0), linestyle="-", color="black", label="Sol")
ax.plot (X_init, (X_init-2)**2 + 1, marker="x", linestyle="none", color="orange", label="Position initiale de la bille")
ax.plot ((X_init,X_init), (0,(X_init-2)**2 + 1), linestyle="-", color="orange", label="Distance initiale")
ax.plot (X, (X-2)**2 + 1, marker="x", linestyle="none", color="green", label="Position de la bille itération 1")
ax.plot ((X,X), (0,(X-2)**2 + 1), linestyle="-", color="green", label="Distance itération 1")
ax.set_title("Forme du récipient")
ax.set_xlabel("Position X")
ax.set_ylabel("Position Y de la bille")
ax.legend()
plt.show()

print ("La nouvelle valeur de X est " + str(X) )

Ca va dans le bon sens, mais ce n'est pas encore parfait... Et si l'on refaisait cela en boucle tant que la dérivée (la pente) de la fonction coût n'est pas assez petite ?

In [ ]:
# Boucle d'optimisation : on avance tant que la pente > eps OU nb itérations max atteint
while abs(grad)>eps:
  gradD = dcout(X)              # Calcul de la pente
  grad = math.sqrt(gradD**2)    # Valeur absolue
  X = X - taux_evolution * gradD        # On bouge un peu X dans la direction opposée de la dérivée
  iter += 1
  if iter > maxIter:
    grad = 0
  # Sauvegarde pour affichage des courbes de pente et distance vs itérations
  liste_iter.append(iter)
  liste_erreur.append(grad)
  liste_dist.append(cout(X))
  liste_X.append(X)

#Affichage de l'évolution du gradient
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (liste_iter,liste_erreur, linestyle="-", linewidth=1, label="Gradient")
ax.set_xlabel("Itérations")
ax.set_ylabel("Valeur gradient du coût")
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.show()

#Affichage de l'évolution de la distance
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (liste_iter,liste_dist, linestyle="-", linewidth=1, label="Distance")
ax.set_xlabel("Itérations")
ax.set_ylabel("Valeur du coût (=distance)")
ax.set_xscale('log')
ax.legend()
plt.show()

print ("Le coût final obtenu est " + str(cout(X)))

La courbe de pente du coût semble converger vers 0, c'est bon signe ! Que donne le modèle obtenu en fin d'optimisation ?

In [ ]:
# Analyse de la position obtenue pour notre optimisation en fin d'optimisation
Y_mod = (X-2)**2 + 1

# Affichage des résultats du modèle
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (Xv, Yv, linestyle="-", linewidth=1, label="Récipient")
ax.plot ((0,4), (0,0), linestyle="-", color="black", label="Sol")
ax.plot (X_init, (X_init-2)**2 + 1, marker="x", linestyle="none", color="orange", label="Position initiale de la bille")
ax.plot ((X_init,X_init), (0,(X_init-2)**2 + 1), linestyle="-", color="orange", label="Distance initiale")
for i in range(0,N):      # Affichage des positions pour chaque itération
  ax.plot (liste_X[i], liste_dist[i], marker="x", linestyle="none", color="blue")
ax.plot (X, (X-2)**2 + 1, marker="x", linestyle="none", color="green", label="Position finale de la bille")
ax.plot ((X,X), (0,(X-2)**2 + 1), linestyle="-", color="green", label="Distance finale")
ax.set_title("Forme du récipient")
ax.set_xlabel("Position X")
ax.set_ylabel("Position Y de la bille")
ax.legend()
plt.show()

print ("Théoriquement on devrait obtenir : X_th = " + str(2) + " et Distance_th = " + str(1))
print ("Le modèle donne en " + str(iter) + " itérations : X_mod = " + str(X) + " et Distance_mod = " + str(cout(X)))


Ca parait magique, non ? Et ça fonctionne bien entendu pour des choses un peu plus complexes, comme nous allons le voir par la suite.

Expliquer pourquoi la distance ne converge pas vers 0.

Que se passe-t-il si l'on met un taux d'évolution de 0.2 ? 0.3 ?... jusqu'à 1 ? Relever à chaque fois le nombre d'itérations nécessaires et le mettre dans un tableau. Conclure.

Et en pratique, ça peut servir à quelque chose ?
=> [Lien vers la partie 2 : Caractérisation d'une batterie](https://colab.research.google.com/drive/1JXGgboVAfwCTm1tow6axvQS3vtgH3vH3#scrollTo=xYl_UBCdxmOC)